## TUIA - Aprendizaje Automático 1

### Trabajo Práctico: Predicción de lluvia en Australia.

### Integrantes:
- Ponce, Daniel
- Yañez, Mirian



El presente informe detalla el trabajo práctico llevado a cabo para la predicción de las condiciones climáticas en Australia, centrándonos en las ciudades de Adelaide, Canberra, Cobar, Dartmoor, Melbourne, MelbourneAirport, MountGambier, Sydney y SydneyAirport. 

El conjunto de datos utilizado se denomina weatherAUS.csv y contiene información climática de los últimos diez años.

## Variables de Interés:

**RainTomorrow y RainfallTomorrow**: Estas variables representan nuestro objetivo de predicción, indicando si lloverá al día siguiente y la cantidad de lluvia, respectivamente.

"RainTomorrow" (categórica, para un problema de clasificación)

"RainfallTomorrow" (continua, para un problema de regresión).

## Paquetes y Librerias

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

## Importar el dataset

In [2]:
file_path= 'weatherAUS.csv'
df = pd.read_csv(file_path, sep=',')

## Filtramos por las ciudades de interés

In [3]:
ciudades = ['Adelaide', 'Canberra', 'Cobar', 'Dartmoor', 'Melbourne', 'MelbourneAirport', 'MountGambier', 'Sydney', 'SydneyAirport']
df = df[df['Location'].isin(ciudades)]

## Realizamos el split de entrenamiento y prueba

In [4]:
train, test = train_test_split(df, test_size = 0.20)

## Análisis descriptivo

Se realizará un análisis exploratorio del conjunto de datos de entrenamiento para entender sus características principales y determinar si se requiere alguna acción para abordar datos faltantes, valores atípicos, la codificación de variables categóricas u otros procesos antes de proceder.

In [5]:
train.columns

Index(['Unnamed: 0', 'Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall',
       'Evaporation', 'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am',
       'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'RainToday', 'RainTomorrow', 'RainfallTomorrow'],
      dtype='object')

In [6]:
train.head()

,Unnamed: 0,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,RainfallTomorrow
35667,35677,2015-02-25,SydneyAirport,19.7,23.6,0.0,6.6,9.1,S,50.0,...,85.0,1015.6,1015.2,7.0,7.0,21.3,20.8,No,Yes,11.4
98276,98307,2014-03-08,Adelaide,15.5,31.4,NaN,NaN,11.5,ESE,41.0,...,28.0,1020.3,1016.1,NaN,NaN,20.9,30.5,NaN,NaN,NaN
65427,65447,2012-07-10,MelbourneAirport,6.8,12.9,0.2,1.8,0.0,N,54.0,...,81.0,1017.8,1011.2,8.0,8.0,10.0,12.6,No,Yes,1.4
82425,82451,2013-03-23,Dartmoor,13.1,23.1,0.2,3.0,6.4,W,48.0,...,50.0,1014.8,1013.3,NaN,NaN,14.8,22.1,No,Yes,2.2
45803,45817,2008-06-18,Canberra,0.4,15.5,0.0,1.0,4.7,N,13.0,...,70.0,1026.7,1023.5,5.0,7.0,4.0,15.0,No,No,0.0


In [7]:
train.info()
train.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 22586 entries, 35667 to 33424
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        22586 non-null  int64  
 1   Date              22586 non-null  object 
 2   Location          22586 non-null  object 
 3   MinTemp           22123 non-null  float64
 4   MaxTemp           22132 non-null  float64
 5   Rainfall          21796 non-null  float64
 6   Evaporation       19134 non-null  float64
 7   Sunshine          17256 non-null  float64
 8   WindGustDir       21264 non-null  object 
 9   WindGustSpeed     21266 non-null  float64
 10  WindDir9am        21232 non-null  object 
 11  WindDir3pm        22231 non-null  object 
 12  WindSpeed9am      22298 non-null  float64
 13  WindSpeed3pm      22309 non-null  float64
 14  Humidity9am       21822 non-null  float64
 15  Humidity3pm       21865 non-null  float64
 16  Pressure9am       21908 non-null  float64

Unnamed: 0             0
Date                   0
Location               0
MinTemp              463
MaxTemp              454
Rainfall             790
Evaporation         3452
Sunshine            5330
WindGustDir         1322
WindGustSpeed       1320
WindDir9am          1354
WindDir3pm           355
WindSpeed9am         288
WindSpeed3pm         277
Humidity9am          764
Humidity3pm          721
Pressure9am          678
Pressure3pm          667
Cloud9am            7513
Cloud3pm            7666
Temp9am              475
Temp3pm              469
RainToday            790
RainTomorrow         794
RainfallTomorrow     794
dtype: int64

### **Hay un total de 22586 datos de entrenamiento, 25 columnas y se puede observar que hay datos nulos en la mayoria de las variables.**

### Eliminamos la columna 'Unnamed: 0'

In [8]:
train = train.drop(columns=['Unnamed: 0'])

### Convertimos a formato fecha la columna Date

In [9]:
train['Date'] = pd.to_datetime(train['Date'])

In [20]:
def rellenar_nulos(row):
    fecha = row['Date']
    valor = row['Sunshine']
    
    if pd.isnull(valor):
        # Buscar otro registro con la misma fecha
        mismo_dia = train[train['Date'] == fecha]
        mismo_dia = mismo_dia.dropna(subset=['Sunshine'])  # Eliminar filas donde la columna_a_rellenar sea nula
        
        if len(mismo_dia) > 0:
            # Tomar el valor de la columna_a_rellenar del primer registro con la misma fecha
            return mismo_dia.iloc[0]['Sunshine']
        else:
            print('acaa')
            return f"No hay registros con la misma fecha {fecha}"
    else:
        return valor
            
def rellenar_nulos_con_fecha_anterior(train, columna_a_rellenar):
    """
    Rellena los valores nulos en la columna_a_rellenar con el valor correspondiente de otra fila que tenga la misma fecha.
    Si no hay ningún otro registro con la misma fecha, retorna un mensaje indicando que no se pudo completar la operación.

    Argumentos:
    - df: DataFrame de pandas.
    - columna_a_rellenar: Nombre de la columna que se rellenará si sus valores son nulos.

    Retorna:
    - DataFrame con los valores nulos rellenados.
    """
    # Ordenar el DataFrame por la columna de fecha
    train = train.sort_values(by='Date')
    
    # Aplicar la función a cada fila
    train[columna_a_rellenar] = train.apply(rellenar_nulos, axis=1)
    
    return train


df_resultado = rellenar_nulos_con_fecha_anterior(train, 'Sunshine')
print(df_resultado)


             Date          Location  MinTemp  MaxTemp  Rainfall  Evaporation  \
45573  2007-11-01          Canberra      8.0     24.3       0.0          3.4   
45574  2007-11-02          Canberra     14.0     26.9       3.6          4.4   
45575  2007-11-03          Canberra     13.7     23.4       3.6          5.8   
45576  2007-11-04          Canberra     13.3     15.5      39.8          7.2   
45577  2007-11-05          Canberra      7.6     16.1       2.8          5.6   
...           ...               ...      ...      ...       ...          ...   
102519 2017-06-24      MountGambier      4.8     12.3       2.2          NaN   
83979  2017-06-24          Dartmoor      6.2     11.8       3.0          NaN   
99480  2017-06-24          Adelaide      5.0     17.3       0.0          NaN   
67178  2017-06-24  MelbourneAirport      5.7     12.6       1.2          1.8   
70370  2017-06-24         Melbourne      8.3     12.3       1.4          1.8   

        Sunshine WindGustDir  WindGustS

In [22]:
df_resultado.isna().sum()

Date                   0
Location               0
MinTemp              463
MaxTemp              454
Rainfall             790
Evaporation         3452
Sunshine               0
WindGustDir         1322
WindGustSpeed       1320
WindDir9am          1354
WindDir3pm           355
WindSpeed9am         288
WindSpeed3pm         277
Humidity9am          764
Humidity3pm          721
Pressure9am          678
Pressure3pm          667
Cloud9am            7513
Cloud3pm            7666
Temp9am              475
Temp3pm              469
RainToday            790
RainTomorrow         794
RainfallTomorrow     794
dtype: int64